In [3]:
%pip install -U google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 5.5 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 7.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23/23 [google-generativeai]ogle-generativeai]language]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import csv
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

#modelo utilizado
MODEL = "gemini-2.5-flash"


## PROMPT ZERO SHOT

In [ ]:
def prompt_zero_shot(texto):
    return f"""
Tarefa: Identificar falácias lógicas em textos argumentativos.

Instruções:
Dado o texto abaixo, identifique quais as seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho

Indique em uma linha de texto de no máximo 120 palavras a resposta que deve conter:
- As falácias (nomes).
- Indique em qual/quais trechos do texto argumentativo as falácias aparecem.
- Dê uma breve explicação do por que aquele trecho representa essa falácia.

Texto:
\"\"\"{texto}\"\"\"

Resposta:
"""

def main():
    model = genai.GenerativeModel(MODEL)
    ids_escolhidos = ["19"]
    exemplos = []
    falacias_lista = [
    "Ad Hominem",
    "Ladeira Escorregadia",
    "Generalização Apressada",
    "Falso Dilema",
    "Apelo à Emoção",
    "Espantalho"
    ]

    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader: 
            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]
            })
        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]
        nome_ids = "_".join(ids_escolhidos)
        nome_arquivo = f"zero_shot_respostas_ids_{nome_ids}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo): #verifica se o arquivo ja existe, se existir cria um com _novo, usamos isso pra testes com mesmo numero de shots pra ver o quanto variam as respostas
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto","falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_zero_shot(texto)
            response = model.generate_content(prompt)
            resposta_texto = " ".join(response.text.split()).strip() #pra deixar formatado no csv
            
            #pra fazer a coluna de falacias encontradas no csv
            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)
            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste zero-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

if __name__ == "__main__":
    main()


Feito teste zero-shot com texto ID: 19
Texto: Os acontecimentos no Nepal e agora no México mostram de forma clara que ou o Brasil regula as redes sociais ou seremos alvo de um golpe digital, tal qual está ocorrendo pelo mundo todo.
Falácias: Ladeira Escorregadia, Generalização Apressada, Falso Dilema, Apelo à Emoção
Resposta: O texto apresenta **Falso Dilema**, **Ladeira Escorregadia**, **Generalização Apressada** e **Apelo à Emoção**. O **Falso Dilema** e a **Ladeira Escorregadia** aparecem em "ou o Brasil regula as redes sociais ou seremos alvo de um golpe digital". É um falso dilema por apresentar apenas duas opções extremas como as únicas existentes. É ladeira escorregadia por sugerir que a não regulação *inevitavelmente* levará a um "golpe digital" sem evidência dos passos intermediários. A **Generalização Apressada** está em "Os acontecimentos no Nepal e agora no México mostram... tal qual está ocorrendo pelo mundo todo", inferindo uma tendência global de apenas dois exemplos. O

## PROMPT FEW-SHOT

In [ ]:
def prompt_few_shot(texto,exemplos_fewshot):
    exemplos_formatados = " "

    for ex in exemplos_fewshot:
        exemplos_formatados += (
            f"Exemplo:\n"
            f"Texto: \"{ex['texto']}\"\n"
            f"Falacias encontradas: {ex['falacia']} \n"
            f"Resposta: {ex['resposta']}\n\n"
        )

    # 🔍 PRINT PARA DEBUGAR OS EXEMPLOS FEW-SHOT
    print("\n===== EXEMPLOS FEW-SHOT FORMATADOS =====")
    print(exemplos_formatados)
    print("========================================\n")

    return f"""

Tarefa: Identificar falácias lógicas em textos argumentativos.

A seguir estão alguns exemplos de como identificar falácias lógicas em textos.

{exemplos_formatados}

Segue as instruções para o próximo texto.

Instruções:
Dado o texto abaixo, identifique quais as seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho

Indique em uma linha de texto de no máximo 120 palavras a resposta que deve conter:
- As falácias (nomes).
- Indique em qual/quais trechos do texto argumentativo as falácias aparecem.
- Dê uma breve explicação do por que aquele trecho representa essa falácia.

Texto:
\"\"\"{texto}\"\"\"

Resposta:
"""

def main():
    model = genai.GenerativeModel(MODEL)

    ids_escolhidos = ["19"]  #escolha dos exemplos pra teste few-shot
    exemplos = []
    ids_exemplos_fewshot = ["12","13","14","3","6"]  #exemplos que irao ser usados no prompt few-shot
    exemplos_fewshot = []

    falacias_lista = [
        "Ad Hominem",
        "Ladeira Escorregadia",
        "Generalização Apressada",
        "Falso Dilema",
        "Apelo à Emoção",
        "Espantalho"
    ]

    #le csv
    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader:
            if coluna["id"] in ids_exemplos_fewshot:
                exemplos_fewshot.append({
                    "id": coluna["id"],
                    "texto": coluna["texto"],
                    "falacia": coluna["falacias_esperadas"], 
                    "resposta": coluna["resposta_esperada"]
                })

            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]    
            })

        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]

        nome_ids = "_".join(ids_escolhidos)
        nome_arquivo = f"few_shot_respostas_ids_{nome_ids}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo):
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto", "falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_few_shot(texto, exemplos_fewshot)
            response = model.generate_content(prompt)

            resposta_texto = " ".join(response.text.split()).strip()

            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)

            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste few-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

            
if __name__ == "__main__":
    main()


===== EXEMPLOS FEW-SHOT FORMATADOS =====
 Exemplo:
Texto: "Se você ama de verdade o Brasil e se importa com o futuro dos seus filhos, precisa protestar contra essa PEC que tenta blindar os parlamentares. Ficar calado agora é trair a nação e deixar que destruam o país."
Falacias encontradas: Apelo à Emoção 
Resposta: O argumento apela ao patriotismo e o amor pelos filhos dos leitores (Apelo a Emoção), tentando manipular suas emoções para que se sintam culpados por não protestar contra a PEC de blindagem aos parlamentares.

Exemplo:
Texto: "Ou você apoia a reforma agraria, ou está contra o progresso e contra os direitos dos trabalhadores."
Falacias encontradas: Falso Dilema 
Resposta: O argumento apresenta apenas duas opções totalmente extremas (Falso Dilema), ignorando a possibilidade de haver outras posições intermediárias ou alternativas sobre a questão da reforma agrária.

Exemplo:
Texto: "Quem defende que o governo deve investir mais em ciência está apenas querendo gastar dinheiro 

## PROMPT ZERO-SHOT-CHAIN-OF-THOUGHT

In [32]:
def prompt_zero_shot_CoT(texto):
    return f"""
Tarefa: Identificar falácias lógicas em textos argumentativos.

Instruções:
Dado o texto abaixo, identifique quais as seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho

Indique em uma linha de texto de no máximo 120 palavras a resposta que deve conter:
- As falácias (nomes).
- Indique em qual/quais trechos do texto argumentativo as falácias aparecem.
- Dê uma breve explicação do por que aquele trecho representa essa falácia.

Texto:
\"\"\"{texto}\"\"\"

Resposta: 
(Pense passo a passo para identificar, classificar e explicar as falácias presentes no texto acima.)
(Mostre APENAS a resposta final, curta e organizada. Não revele o raciocínio.)

"""

def main():
    model = genai.GenerativeModel(MODEL)
    ids_escolhidos = ["19"]
    exemplos = []
    falacias_lista = [
    "Ad Hominem",
    "Ladeira Escorregadia",
    "Generalização Apressada",
    "Falso Dilema",
    "Apelo à Emoção",
    "Espantalho"
    ]

    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader: 
            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]
            })
        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]
        nome_ids = "_".join(ids_escolhidos)
        nome_arquivo = f"zero_shot_cot_respostas_ids_{nome_ids}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo): #verifica se o arquivo ja existe, se existir cria um com _novo, usamos isso pra testes com mesmo numero de shots pra ver o quanto variam as respostas
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto","falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_zero_shot(texto)
            response = model.generate_content(prompt)
            resposta_texto = " ".join(response.text.split()).strip() #pra deixar formatado no csv
            
            #pra fazer a coluna de falacias encontradas no csv
            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)
            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste zero-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

if __name__ == "__main__":
    main()


Feito teste zero-shot com texto ID: 19
Texto: Os acontecimentos no Nepal e agora no México mostram de forma clara que ou o Brasil regula as redes sociais ou seremos alvo de um golpe digital, tal qual está ocorrendo pelo mundo todo.
Falácias: Generalização Apressada, Falso Dilema, Apelo à Emoção
Resposta: As falácias presentes são Generalização Apressada, Falso Dilema e Apelo à Emoção. A **Generalização Apressada** surge ao usar apenas os casos do Nepal e México para uma conclusão abrangente sobre o Brasil e o mundo ("Os acontecimentos no Nepal e agora no México mostram de forma clara que..."). O **Falso Dilema** é evidente em "ou o Brasil regula as redes sociais ou seremos alvo de um golpe digital", que limita as opções a apenas duas extremas, ignorando outras possibilidades. O **Apelo à Emoção** se manifesta em "...seremos alvo de um golpe digital...", explorando o medo de um evento catastrófico para forçar a aceitação do argumento, em vez de uma justificação lógica.

